In [1]:
import pandas as pd
import pickle
import numpy as np
from pathlib import Path

mat_path = Path('./Dataset/matches_csv')

score_path = Path('./experiment/matches_scores')
models = [model.name for model in list(score_path.glob('*'))]

save_path = Path('./experiment/scores_with_matdata')

In [2]:
def connect(match_list, model):
    for idx, match in enumerate(match_list):
        endstr = '\n' if idx == len(match_list)-1 else '\r'
        print(f'[{model} results] ({idx+1:5d}/{len(match_list)}) {(idx+1)/len(match_list)*100:3.1f}% progressing...', end=endstr)
        mat_csv = mat_path / f'{match.name[:-4]}.csv'

        mat_csv = pd.read_csv(mat_csv)
        mat_csv.insert(len(mat_csv.columns), 'score', np.nan)

        with match.open('rb') as file: mat_sco = pickle.load(file)

        for player in range(10):
            if (player+1) not in list(mat_csv.player): continue
            scos = mat_sco[player].squeeze().tolist()
            if isinstance(scos, list): scos.reverse()
            mat_csv.loc[(mat_csv.player==player+1), 'score'] = scos

        save_path = Path('./experiment/scores_with_matdata') / model
        if not save_path.exists(): save_path.mkdir(parents=True)
        mat_csv.to_feather(save_path / f'{match.name[:-4]}.ftr')

In [3]:
for model in models:
    path = score_path / model
    connect(list(path.glob('*')), model)

[normal results] (45575/45575) 100.0% progressing...
[normal_bce results] (45575/45575) 100.0% progressing...
[mlp_bce results] (45575/45575) 100.0% progressing...
[zero_h0_bce results] (45575/45575) 100.0% progressing...
[zero_h0_reverse results] (45575/45575) 100.0% progressing...
[zero_h0 results] (45575/45575) 100.0% progressing...
[mlp results] (45575/45575) 100.0% progressing...
